In [5]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi
from scipy.optimize import curve_fit
from scipy.constants import *
from engineering_notation import EngNumber

import BBO

%matplotlib inline
# %matplotlib widget

# Wydajność SPDC
$$
P=\frac{NE}{t}=\frac{hc}{\lambda}\frac{N}{t}
$$

$$
\eta_{pair}=\frac{P_{out/channel}}{P_{in}}
$$
Rozmiar plamki lasera:
$$
w_f=\frac{\lambda f_{lense}}{\pi w_0}
$$
Z doświadczeń dane pokazuję że wydajność wynosi:

In [2]:
P0 = 2.5e-3
l = 0.370  # in um
L = 1e-3
A = np.pi * (0.1 * l * 1e-6 / (np.pi * 1e-3)) ** 2  # na oko
l2 = 2 * l

N0dot = P0 / (h * c / (l * 1e-6))
Ndot = 100e3
NCdot = 150
print(EngNumber(Ndot / N0dot) / 2)
print(EngNumber(NCdot / N0dot) / 2)

10.74p
16.11f


## Obliczanie wydajności
### W oparciu o "Handbook of Nonlinear Optical Crystals" V.M. Dimitrieva

Gdzie $d_{eff}$ jest efektywną podatnością elektryczną, która najogólniej wynosi (eq. 2.46):
$$
P_i=\varepsilon_0 \chi_{ij} E_j + 2d_{ik} E^2_k
$$
W związku symetriami Kleinmana (eq. 2.48) (które działają przy **braku** dyspersji) $d_{31}=d_{15}$, wiedząc że BBO jest kryształem klasy 3 m (3.1.18), to z (tab. 2.3)
$$
d_{eff,BBO}=d_{15}\sin\theta-d_{22}\cos\theta\sin3\varphi
$$
gdzie $\theta$ jest kątem do osi kryształu, a $\varphi$ jest kątem między płaszczyzną główną, a wiązką. Dane z (3.1.18):
$$
d_{22}=(2.22\pm0.09)\times 10^{-12} ~\text{m}/\text{V}
$$
$$
d_{31}=(0.16\pm0.08)\times 10^{-12} ~\text{m}/\text{V}
$$
## Inny sposób
Z (tab. 4.16) dane jest że $d_{eff,BBO}=0.3\times d_{eff,KDP}$, a z 3.1.1 dla KDP (KH $_2$ PO $_4$):
$$d_{eff,KDP}\approx d_{36,KDP}(\lambda = 0.63\mu \text{~m}) = 7.1\times 10^{-13}~\text{m}/\text{V}$$

## Z publikacji
Efficiency of entangled photon pair generation in bulk crystals and waveguides – a comparison  Anthony Leverrier
$$
Rate=\frac{L d_{\text{eff}}^2 i_p \omega _i \omega _s}{2 c^2 \varepsilon_0  n_i n_p
   n_s \left| D \right| }

$$

$$
\eta = \frac{Rate}{\dot{N_0}}
$$
podane jest $d_{eff}=2.02$ pm/V\
https://www.fuw.edu.pl/~zopt/photonics/UFO_PW/UFO_10_SHG_THG_phase_matching.pdf
$$
GVM=D = \frac{1}{v_{g,i}}-\frac{1}{v_{g,s}}\quad v_g=\frac{\partial \omega}{\partial k}=\frac{c}{n-\lambda\frac{\partial n}{\partial \lambda}}

$$
$$
GVM=D=\frac{1}{c}\left| n_i - n_s + \left. \lambda_s\frac{\partial n_s}{\partial \lambda}\right|_{\lambda=\lambda_s}  - \lambda_i\left. \frac{\partial n_i}{\partial \lambda}\right|_{\lambda=\lambda_i}\right|
$$

In [5]:
def nl(x):
    return BBO.neeff(x, np.deg2rad(2))


# [\lambda] = um
def deriv(f, x0, h=1e-9):
    return (f(x0 + h) - f(x0)) / h


def dndl(x):
    return deriv(nl, x)


D = abs(BBO.no(l2) - nl(l) + l * deriv(nl, l) - l2 * deriv(BBO.no, l2))
print(D)


def efficiency2(deff, n1, n2, L, l1, l2, P, A):
    return (
        1
        / (2 * c**2 * epsilon_0)
        * L
        * deff**2
        * (P / A)
        * (2 * pi * c / l1)
        * (2 * pi * c / l1)
        / (n1 * n2 * n1)
        / np.abs(D)
    )


# d_eff max
deffBBO = 2.22e-12  # m/V
print(
    EngNumber(
        efficiency2(deffBBO, BBO.no(2 * l), nl(l), L, l * 1e-6, l2 * 1e-6, P0, A)
        / N0dot
    )
)

# d_eff z publikacji
deffBBO = 2.02e-12  # m/V
print(
    EngNumber(
        efficiency2(deffBBO, BBO.no(2 * l), nl(l), L, l * 1e-6, l2 * 1e-6, P0, A)
        / N0dot
    )
)

# d_eff z Handbook
deffKDP = 7e-13  # m/V
deffBBO = 0.3 * deffKDP
print(
    EngNumber(
        efficiency2(deffBBO, BBO.no(2 * l), nl(l), L, l * 1e-6, l2 * 1e-6, P0, A)
        / N0dot
    )
)

# d_eff min
deffBBO = 0.16e-12  # m/V
print(
    EngNumber(
        efficiency2(deffBBO, BBO.no(2 * l), nl(l), L, l * 1e-6, l2 * 1e-6, P0, A)
        / N0dot
    )
)

0.11644622752518852
180.46p
149.41p
1.61p
937.38f


W "Quantum electronics"
$$
P_2=\pi \frac{\beta L P_3}{|b|}\theta^2
$$
Ale, gdyby przyjąć inny rozkłąd $\nu_s$:

$$
P_s=\frac{1}{2}\beta L^2 P_p\theta^2\nu _s \text{sinc}^2\left(\frac{1}{2}L\Delta k\right)
$$

In [ ]:
from scipy.misc import derivative


def efficiency3(deff):
    fs = c / (2 * l * 1e-6)
    beta = (
        2
        * hbar
        / ((2 * pi) ** 2 * epsilon_0**1 * c**5)
        * fs**5
        * deff**2
        / BBO.neeff(l, np.deg2rad(3))
    )

    # def y(l):
    # return np.sqrt(2.7405 + (0.0184) / (l**2 - 0.0179) - 0.0155 * l**2)
    # GVD = -l * 1e6 / c * derivative(y, l, dx=1e-9, n=2) * 1e-12
    return beta * L**2 * fs * (np.deg2rad(3)) ** 2 / 2


# d_eff max
deffBBO = 2.22e-12  # m/V
print(EngNumber(efficiency3(deffBBO)))

# d_eff z publikacji
deffBBO = 2.02e-12  # m/V-p
print(EngNumber(efficiency3(deffBBO)))

# d_eff z Handbook
deffKDP = 7e-13  # m/V
deffBBO = 0.3 * deffKDP
print(EngNumber(efficiency3(deffBBO)))

# d_eff min
deffBBO = 0.16e-12  # m/V
print(EngNumber(efficiency3(deffBBO)))

10.80y
8.94y


KeyError: '-51'